In [ ]:
%cd ~/ASMSA/trpcage

In [ ]:
threads = 2

import os
os.environ['OMP_NUM_THREADS']=str(threads)
import tensorflow as tf

# PyTorch favours OMP_NUM_THREADS in environment
import torch

# Tensorflow needs explicit cofig calls
tf.config.threading.set_inter_op_parallelism_threads(threads)
tf.config.threading.set_intra_op_parallelism_threads(threads)

In [ ]:
from tensorflow import keras as k
import numpy as np
import asmsa.visualizer as visualizer
import mdtraj as md
import matplotlib.pyplot as plt
import math

In [ ]:
exec(open('inputs.py').read())

In [ ]:
def _compute_number_of_neurons(layers, seed):
    neurons = [seed]

    tmp = seed
    for _ in range(layers-1):
        tmp = int(tmp / 2)
        neurons.append(tmp)
    return neurons

In [ ]:
class AAEClassModel(k.models.Model):
    def __init__(self,
            n_features,
            n_classes,
            enc_layers, enc_seed,
            disc_layers, disc_seed,
            af='gelu',
            bn_momentum=0.8, leak_alpha=0.2,
            latent_dim=2,
            dist_threshold=5.
        ):

        super().__init__()
# https://doi.org/10.48550/arXiv.1511.05644, Sect. 7 / Fig. 10
        self.n_classes = n_classes
        self.latent_dim = latent_dim
        self.dist_threshold2 = dist_threshold*dist_threshold
        
        enc_neurons = _compute_number_of_neurons(enc_layers, enc_seed) 
        disc_neurons = _compute_number_of_neurons(disc_layers, disc_seed)

        inp = k.Input(shape=(n_features,),name='inp')
        out = inp

        for n in range(enc_layers):
            out = k.layers.Dense(enc_neurons[n],activation=af,name=f'enc_{n}')(out)
            out = k.layers.BatchNormalization(momentum=bn_momentum, name=f'enc_bn_{n}')(out)

        z = k.layers.Dense(latent_dim,name='latent_z')(out)
        y = k.layers.Dense(n_classes,activation='softmax',name='latent_y')(out)

        chs = k.layers.Dense(latent_dim,name='cluster_heads')
        hz = chs(y)
        d = (math.pow(n_classes,1./latent_dim) - 1) * dist_threshold / 2 / 3 # XXX
        chs.set_weights([
            np.random.uniform(low=-d,high=d,size=(n_classes,latent_dim)),
            np.random.normal(size=(latent_dim,))
        ])
        
        latent = k.layers.Add(name='add_z_hz')([z,hz])

        out = latent
        for n in reversed(range(enc_layers)):
            out = k.layers.Dense(enc_neurons[n],activation=af,name=f'dec_{n}')(out)
            out = k.layers.BatchNormalization(momentum=bn_momentum, name=f'dec_bn_{n}')(out)

        dec_out = k.layers.Dense(n_features,name='dec_out')(out)

        out = y
        for n in range(disc_layers):
            out = k.layers.Dense(disc_neurons[n],name=f'y_disc_{n}')(out)
            out = k.layers.LeakyReLU(negative_slope=leak_alpha,name=f'y_disc_relu_{n}')(out)
        
        y_disc_out = k.layers.Dense(1,name='y_disc_out')(out)

        out = z
        for n in range(disc_layers):
            out = k.layers.Dense(disc_neurons[n],name=f'z_disc_{n}')(out)
            out = k.layers.LeakyReLU(negative_slope=leak_alpha,name=f'z_disc_relu_{n}')(out)

        z_disc_out = k.layers.Dense(1,name='z_disc_out')(out)

        self.enc = k.Model(inputs=inp,outputs=[y,z])
        self.sum = k.Model(inputs=[y,z],outputs=latent)
        self.dec = k.Model(inputs=latent,outputs=dec_out)
        self.heads = k.Model(inputs=y,outputs=hz)
        # self.ae = k.Model(inputs=inp,outputs=dec_out)
        self.y_disc = k.Model(inputs=y,outputs=y_disc_out)
        self.z_disc = k.Model(inputs=z,outputs=z_disc_out)

    def compile(self,optimizer=None,lr=None):
        
        if optimizer is None:
            opt = k.optimizers.Adam(learning_rate=lr)
        else:
            opt = optimizer

        self.ae_loss = k.losses.MeanSquaredError()

        super().compile(optimizer=opt,loss=self.ae_loss)
        self.optimizer.build(self.enc.trainable_weights+self.sum.trainable_weights+self.dec.trainable_weights+self.y_disc.trainable_weights+self.z_disc.trainable_weights)

        self.enc.compile()
        self.dec.compile()
        self.y_disc.compile()
        self.z_disc.compile()

    @tf.function
    def train_step(self,in_batch):
        if isinstance(in_batch, tuple):
            batch = in_batch[0]
        else:
            batch = in_batch

        # autoencoder
        with tf.GradientTape() as aet:
            y,z = self.enc(batch)
            rec = self.dec(self.sum([y,z]))
            ael = self.ae_loss(batch,rec)
    
        aew = self.enc.trainable_weights + self.sum.trainable_weights + self.dec.trainable_weights
        aeg = aet.gradient(ael,aew)
        self.optimizer.apply_gradients(zip(aeg,aew))
    
        # categoric discriminator
        idx = tf.random.uniform((batch.shape[0],), minval=0, maxval=self.n_classes, dtype=tf.int32)
        randy = tf.one_hot(idx, depth=self.n_classes)
    
        # binary crossentropy from logits
        with tf.GradientTape() as yt:
            nyp = self.y_disc(y)
            nyp *= tf.random.uniform(tf.shape(nyp), 1., 1.05)
            nyl = tf.reduce_mean(nyp,axis=0)
    
            pyp = self.y_disc(randy)
            pyp *= tf.random.uniform(tf.shape(pyp), 1., 1.05)
            pyl = -tf.reduce_mean(pyp,axis=0)
            y_disc_loss = (nyl + pyl) * 1e-5 #XXX
    
        yg = yt.gradient(y_disc_loss,self.y_disc.trainable_weights)
        self.optimizer.apply_gradients(zip(yg,self.y_disc.trainable_weights))
    
        # cheet it
        with tf.GradientTape() as yct:
            yc = self.y_disc(self.enc(batch)[0])
            yc *= tf.random.uniform(tf.shape(yc), 1., 1.05)
            ycl = -tf.reduce_mean(yc, axis=0) * 1e-5 #XXX
    
        ycg = yct.gradient(ycl,self.enc.trainable_weights)
        self.optimizer.apply_gradients(zip(ycg,self.enc.trainable_weights))
            
        # intra category discriminator
        randz = tf.random.normal(shape=(batch.shape[0], self.latent_dim))
    
        with tf.GradientTape() as zt:
            nzp = self.z_disc(z)
            nzp *= tf.random.uniform(tf.shape(nzp), 1., 1.05)
            nzl = tf.reduce_mean(nzp,axis=0)
    
            pzp = self.z_disc(randz)
            pzp *= tf.random.uniform(tf.shape(pzp), 1., 1.05)
            pzl = -tf.reduce_mean(pzp,axis=0)
            z_disc_loss = nzl + pzl
    
        zg = zt.gradient(z_disc_loss,self.z_disc.trainable_weights)
        self.optimizer.apply_gradients(zip(zg,self.z_disc.trainable_weights))
            
        # cheet it
        with tf.GradientTape() as zct:
            zc = self.z_disc(self.enc(batch)[1])
            zc *= tf.random.uniform(tf.shape(zc), 1., 1.05)
            zcl = -tf.reduce_mean(zc, axis=0)
    
        zcg = zct.gradient(zcl,self.enc.trainable_weights)
        self.optimizer.apply_gradients(zip(zcg,self.enc.trainable_weights))
        
        # keep cluster heads apart
        randc = tf.linalg.diag(tf.random.uniform((self.n_classes,),0.95,1.05))
        with tf.GradientTape() as ht:
            ch = self.heads(randc)
            norms = tf.reduce_sum(tf.square(ch), axis=1, keepdims=True)  # shape=(N,1)
            dists_squared = norms - 2 * tf.matmul(ch, ch, transpose_b=True) + tf.transpose(norms)
            small_dists = tf.boolean_mask(tf.exp(-dists_squared), dists_squared < self.dist_threshold2)
            hl = tf.reduce_sum(small_dists)/self.dist_threshold2/self.n_classes/self.n_classes * 21. #XXX

        hg = ht.gradient(hl,self.heads.trainable_weights)
        self.optimizer.apply_gradients(zip(hg,self.heads.trainable_weights))

        return { 
            'ae_loss' : ael,
            'y_disc_loss': y_disc_loss,
            'z_disc_loss': z_disc_loss,
            'y_cheat_loss': ycl,
            'z_cheat_loss': zcl,
            'cluster_head_loss': hl
        }

    @tf.function
    def call(self,inp):
        return self.sum(self.enc(inp))

    def call_enc(self,inp):
        return self.call(inp)

            

In [ ]:
# XXX: essential hps manually
batch_size = 64
best_enc_seed=128
best_disc_seed=128 
ae_layers=3
disc_layers=3
learning_rate=0.00002
n_classes = 40

In [ ]:
X_train = tf.data.Dataset.load('datasets/intcoords/train')

# get batched version of dataset to feed to AAE model for training
X_train_batched = X_train.batch(batch_size,drop_remainder=True)

# get numpy version for visualization purposes
X_train_np = np.stack(list(X_train))
X_train_np.shape

In [ ]:
X_test = tf.data.Dataset.load('datasets/intcoords/test')

# get numpy version for visualization purposes
X_test_np = np.stack(list(X_test))
X_test_np.shape

In [ ]:
m = AAEClassModel(n_features=X_train_np.shape[1],n_classes=n_classes,enc_layers=ae_layers,enc_seed=best_enc_seed,disc_layers=disc_layers,disc_seed=best_disc_seed)

In [ ]:
m.compile(lr=learning_rate)

In [ ]:
m.fit(X_train_batched,epochs=500,verbose=2,
     callbacks = [ visualizer.VisualizeCallback(m,freq=10,inputs=X_test_np,figsize=(12,3)) ])

In [ ]:
visualizer.Visualizer(figsize=(12,3)).make_visualization(m(X_train_np[::10,:]).numpy())

In [ ]:
visualizer.Visualizer(figsize=(12,3)).make_visualization(m(X_test_np).numpy())

In [ ]:
tr = md.load('x_test.xtc',top=conf)

In [ ]:
lows = m(X_test_np).numpy()
rg = md.compute_rg(tr)
base = md.load(conf)
rmsd = md.rmsd(tr,base[0])
cmap = plt.get_cmap('rainbow')
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.scatter(lows[:,0],lows[:,1],marker='.',c=rg,cmap=cmap,s=1)
plt.colorbar(cmap=cmap)
plt.title("Rg")
plt.subplot(122)
plt.scatter(lows[:,0],lows[:,1],marker='.',c=rmsd,cmap=cmap,s=1)
plt.colorbar(cmap=cmap)
plt.title("RMSD")
plt.show()

In [ ]:
heads = m.heads(tf.linalg.diag(tf.constant([1.] * n_classes))).numpy()
c = np.log(np.sum(m.enc(X_test_np)[0].numpy(),axis=0))
plt.scatter(heads[:,0],heads[:,1],c=c,cmap='magma')
plt.colorbar()
plt.show()

In [ ]:
z = m.enc(X_test_np)[1].numpy()
plt.scatter(z[:,0],z[:,1])
plt.show()